In [31]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime

In [69]:
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_190615.txt'
df = pd.read_csv(url, header = 0, sep=',')
df.columns = map(str.lower, df.columns) # - make headers lowercase
df.columns = df.columns.str.strip().str.lower()

In [47]:
df.head(5)

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,datetime,exit_increments,entry_increments,previous_entry_number,previous_datetime
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457,2019-06-22 00:00:00,-1,-1,-1,1980-01-01 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465,2019-06-22 04:00:00,8,13,7107725,2019-06-22 00:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491,2019-06-22 08:00:00,26,23,7107738,2019-06-22 04:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541,2019-06-22 12:00:00,50,97,7107761,2019-06-22 08:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581,2019-06-22 16:00:00,40,217,7107858,2019-06-22 12:00:00


In [70]:
df['ttime'] = pd.to_datetime(df['time'])
df['tdate'] = pd.to_datetime(df['date'])
df['datetime'] = df.apply (lambda x: datetime.datetime (x['tdate'].year, x['tdate'].month, x['tdate'].day,
                                                       x['ttime'].hour), axis=1)
df.drop (['ttime','tdate'], inplace=True, axis=1)
df['exit_increments'] = df['exits']
df['entry_increments'] = df['entries']
df['previous_entry_number'] = df['entries']
df['previous_datetime'] = df['datetime']

In [19]:
print (df)

          c/a  unit       scp        station linename division        date  \
0        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
1        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
2        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
3        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
4        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
5        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/22/2019   
6        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/23/2019   
7        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/23/2019   
8        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/23/2019   
9        A002  R051  02-00-00          59 ST  NQR456W      BMT  06/23/2019   
10       A002  R051  02-00-00          59 ST  NQR456W      BMT  06/23/2019   
11       A002  R051  02-00-00          59 ST  NQR456W      BMT  

In [71]:
def max_minus_min_exits (x):
    first_iteration = True
    previous_exits = 0
    for index, row in x.iteritems():
        if first_iteration:
            first_iteration = False
            previous_exits = row
            df.at [index, 'exit_increments'] = -1
        else:
            df.at [index, 'exit_increments'] = row - previous_exits
            previous_exits = row
    result = x.max()-x.min()
    return result
    
def max_minus_min_entries (x):
    first_iteration = True
    previous_entries = 0
    previous_datetime = datetime.datetime (1980,1,1)
    for index, row in x.iteritems():
        if first_iteration:
            first_iteration = False
            previous_entries = row
            df.at [index, 'previous_datetime'] = previous_datetime
            previous_datetime = df.at [index, 'datetime']
            df.at [index, 'entry_increments'] = -1
            df.at [index, 'previous_entry_number'] = -1
        else:
            df.at [index, 'previous_datetime'] = previous_datetime
            previous_datetime = df.at [index, 'datetime']
            df.at [index, 'entry_increments'] = row - previous_entries
            df.at [index, 'previous_entry_number'] = previous_entries
            previous_entries = row
    result = x.max()-x.min()
    return result
    
dfgb_exit_group = df.groupby (['station', 'c/a', 'unit', 'scp']) ['exits'].apply (max_minus_min_exits)
dfgb_entry_group = df.groupby (['station', 'c/a', 'unit', 'scp']) ['entries'].apply (max_minus_min_entries)
dfgb2 = dfgb_exit_group.groupby ('station')
dfgb3 = dfgb2.apply (lambda x:x.sum())
dfgb4 = dfgb3.sort_values (0, False)
#print (dfgb4)
#print (dfgb4.sum())
#print(len(dfgb4))
dfgb4.tail(360).sum()
dfgb=dfgb_exit_group.sort_values (0, False)
print (dfgb.sum())
print (len(dfgb))
print (dfgb)
dfgb.tail(4875).sum()

1935980727
4876
station          c/a    unit  scp     
167 ST           N207   R104  00-00-00    1821597417
59 ST            C021   R212  00-00-03      14611973
42 ST-PORT AUTH  N062A  R010  00-06-01       4576563
23 ST            N077   R111  02-06-02       3130109
CHRISTOPHER ST   PTH09  R548  00-00-02       2928014
EXCHANGE PLACE   PTH05  R543  00-00-07       2846786
THIRTY ST        PTH13  R541  00-04-05       2844815
PAVONIA/NEWPORT  PTH06  R546  00-00-05       2611206
THIRTY THIRD ST  PTH17  R541  01-00-05       2221453
TWENTY THIRD ST  PTH12  R542  00-04-00       2111768
PAVONIA/NEWPORT  PTH06  R546  00-00-06       2012831
EXCHANGE PLACE   PTH05  R543  00-00-06       1911541
PAVONIA/NEWPORT  PTH06  R546  00-00-08       1878563
174-175 STS      N212   R253  01-00-01       1789843
THIRTY THIRD ST  PTH17  R541  01-00-02       1783205
                              01-00-09       1591311
EXCHANGE PLACE   PTH05  R543  00-00-04       1349652
LACKAWANNA       PTH16  R550  01-02-00      

114383310

In [76]:
dfgb

station          c/a    unit  scp     
52 ST            R521   R327  00-06-00    867827588
SPRING ST        N087   R282  01-06-01    117390566
8 AV             H001   R175  00-06-00     56432969
BAY PKWY         D010   R394  00-00-03      5328704
BURNSIDE AV      R287   R244  00-00-00      2028951
8 ST-NYU         A038   R085  00-06-00       119142
WALL ST          R204   R043  02-06-00       102846
GRD CNTRL-42 ST  R238A  R046  02-00-04        56193
                              02-03-00        51842
                              02-00-02        49589
                              02-03-01        45038
                              02-03-02        44010
                              02-00-03        43621
PATH NEW WTC     PTH22  R540  00-01-00        43043
34 ST-HERALD SQ  A022   R022  01-00-00        42525
GRD CNTRL-42 ST  R238A  R046  02-00-01        41512
57 ST-7 AV       A010   R080  00-00-07        38584
W 4 ST-WASH SQ   N083   R138  01-06-01        38084
34 ST-HERALD SQ  A022   R

In [72]:
df2=df.drop(['linename','date','time','exits','division','desc'], axis=1)
print (len(df2.index))
df2=df2.drop(df2[df2.entry_increments < 1].index)
df2=df2.drop(df2[df2.station != 'GRD CNTRL-42 ST'].index)
print(df2)
df2.to_csv('/Users/skydivedavis/Documents/gct2.csv')

204934
          c/a  unit       scp          station  entries            datetime  \
157567   R236  R045  00-00-00  GRD CNTRL-42 ST  2022541 2019-06-08 09:00:00   
157568   R236  R045  00-00-00  GRD CNTRL-42 ST  2022671 2019-06-08 13:00:00   
157569   R236  R045  00-00-00  GRD CNTRL-42 ST  2022906 2019-06-08 17:00:00   
157570   R236  R045  00-00-00  GRD CNTRL-42 ST  2023080 2019-06-08 21:00:00   
157571   R236  R045  00-00-00  GRD CNTRL-42 ST  2023147 2019-06-09 05:00:00   
157572   R236  R045  00-00-00  GRD CNTRL-42 ST  2023167 2019-06-09 09:00:00   
157573   R236  R045  00-00-00  GRD CNTRL-42 ST  2023201 2019-06-09 13:00:00   
157574   R236  R045  00-00-00  GRD CNTRL-42 ST  2023308 2019-06-09 17:00:00   
157575   R236  R045  00-00-00  GRD CNTRL-42 ST  2023426 2019-06-09 21:00:00   
157576   R236  R045  00-00-00  GRD CNTRL-42 ST  2023467 2019-06-10 01:00:00   
157578   R236  R045  00-00-00  GRD CNTRL-42 ST  2023531 2019-06-10 09:00:00   
157579   R236  R045  00-00-00  GRD CNTRL-42 S

In [65]:
print(df2)
df2.to_csv('/Users/skydivedavis/Documents/gct.csv')

          c/a  unit       scp          station    entries            datetime  \
157648   R236  R045  00-00-02  GRD CNTRL-42 ST   10486054 2019-06-08 05:00:00   
157595   R236  R045  00-00-00  GRD CNTRL-42 ST    2029998 2019-06-13 05:00:00   
157746   R236  R045  00-00-04  GRD CNTRL-42 ST   12360879 2019-06-11 01:00:00   
158738   R238  R046  00-00-04  GRD CNTRL-42 ST     897296 2019-06-14 04:00:00   
157677   R236  R045  00-00-02  GRD CNTRL-42 ST   10491650 2019-06-13 05:00:00   
157659   R236  R045  00-00-02  GRD CNTRL-42 ST   10486547 2019-06-10 05:00:00   
157718   R236  R045  00-00-03  GRD CNTRL-42 ST    4008277 2019-06-13 05:00:00   
159082   R238  R046  00-05-00  GRD CNTRL-42 ST   34211474 2019-06-08 12:00:00   
159080   R238  R046  00-05-00  GRD CNTRL-42 ST   34211473 2019-06-08 04:00:00   
158919   R238  R046  00-03-01  GRD CNTRL-42 ST   21079908 2019-06-09 08:00:00   
158534   R238  R046  00-00-00  GRD CNTRL-42 ST      85724 2019-06-08 04:00:00   
158284   R237  R046  01-00-0

In [ ]:
def busiest_